In [ ]:
from credentials import credentials

import requests
import pandas as pd
import time

In [ ]:
centers_switzerland = [
    "Zurich, Switzerland",
    "Geneva, Switzerland",
    "Basel, Switzerland",
    "Lausanne, Switzerland",
    "Bern, Switzerland",
    "Winterthur, Switzerland",
    "Lucerne, Switzerland",
    "St. Gallen, Switzerland",
    "Lugano, Switzerland",
    "Biel/Bienne, Switzerland",
    "Thun, Switzerland",
    "Köniz, Switzerland",
    "La Chaux-de-Fonds, Switzerland",
    "Schaffhausen, Switzerland",
    "Fribourg, Switzerland",
    "Chur, Switzerland",
    "Neuchâtel, Switzerland",
    "Uster, Switzerland",
    "Sion, Switzerland"
]

centers_germany = [
    "Berlin, Germany",
    "Hamburg, Germany",
    "Munich, Germany",
    "Cologne, Germany",
    "Frankfurt, Germany",
    "Stuttgart, Germany",
    "Düsseldorf, Germany",
    "Dortmund, Germany",
    "Essen, Germany",
    "Leipzig, Germany",
    "Bremen, Germany",
    "Dresden, Germany",
    "Hanover, Germany",
    "Nuremberg, Germany",
    "Duisburg, Germany",
    "Bochum, Germany",
    "Wuppertal, Germany",
    "Bielefeld, Germany",
    "Bonn, Germany",
    "Münster, Germany",
    "Karlsruhe, Germany",
    "Mannheim, Germany",
    "Augsburg, Germany",
    "Wiesbaden, Germany",
    "Gelsenkirchen, Germany",
    "Mönchengladbach, Germany",
    "Braunschweig, Germany",
    "Chemnitz, Germany",
    "Kiel, Germany",
    "Aachen, Germany",
    "Halle, Germany",
    "Magdeburg, Germany",
    "Freiburg, Germany",
    "Krefeld, Germany",
    "Lübeck, Germany",
    "Oberhausen, Germany",
    "Erfurt, Germany",
    "Mainz, Germany",
    "Rostock, Germany",
    "Kassel, Germany"
]

In [ ]:
# Google Maps API Key
API_KEY = credentials['api_key']

# Google Places and Place Details endpoint URLs
BASE_PLACES_URL = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
PLACE_DETAILS_URL = 'https://maps.googleapis.com/maps/api/place/details/json'


def get_coordinates_for_city(city_name):
    """Fetch the latitude and longitude for a given city using Google's Geocoding API."""
    print(city_name)
    
    params = {
        'address': city_name,
        'key': API_KEY
    }
    
    response = requests.get(BASE_GEOCODING_URL, params=params)
    location_data = response.json()
    print(location_data)
    location = location_data['results'][0]['geometry']['location']
    
    return location['lat'], location['lng']


def get_place_details(place_id):
    """Retrieve detailed information for a given place using its place_id."""
    
    params = {
        'place_id': place_id,
        'fields': 'name,address_component,formatted_phone_number,website',
        'key': API_KEY
    }
    
    response = requests.get(PLACE_DETAILS_URL, params=params)
    
    return response.json().get('result', {})


def get_boutiques_for_center(lat, lng):
    """Fetch boutiques near a given latitude and longitude."""
    
    boutiques = []
    params = {
        'location': f'{lat},{lng}',
        'radius': 15000,
        'type': 'store',
        'keyword': 'boutique',
        'key': API_KEY
    }
    
    response = requests.get(BASE_PLACES_URL, params=params)
    results = response.json()
    
    boutiques.extend(results.get('results', []))
    
    # Get additional results from next pages (if they exist)
    while 'next_page_token' in results:
        time.sleep(2)  # Wait a bit before next request to ensure the token is valid
        params['pagetoken'] = results['next_page_token']
        response = requests.get(BASE_PLACES_URL, params=params)
        results = response.json()
        boutiques.extend(results.get('results', []))
    
    return boutiques


def process_centers(centers):
    """Retrieve boutiques details for a list of city centers."""
    
    all_boutiques = []
    
    for center in centers:
        lat, lng = get_coordinates_for_city(center)
        boutiques = get_boutiques_for_center(lat, lng)
        
        for boutique in boutiques:
            details = get_place_details(boutique['place_id'])

            # Extracting required information
            name = details.get('name', '')
            address = ''.join([component['long_name'] + ' ' for component in details.get('address_components', []) if component['types'][0] in ['route', 'street_number']]).strip()
            postal_code = ''.join([component['long_name'] for component in details.get('address_components', []) if 'postal_code' in component['types']])
            phone_number = details.get('formatted_phone_number', '')
            website = details.get('website', '')

            all_boutiques.append({
                'Name': name,
                'Address': address,
                'PLZ': postal_code,
                'Phone Number': phone_number,
                'Website': website
            })

    return all_boutiques


# Process Switzerland and Germany centers separately and save results to Excel
df_switzerland = pd.DataFrame(process_centers(centers_switzerland)).drop_duplicates()
df_switzerland.to_excel('swiss_boutiques.xlsx', index=False)

df_germany = pd.DataFrame(process_centers(centers_germany)).drop_duplicates()
df_germany.to_excel('german_boutiques.xlsx', index=False)


In [9]:
import pandas as pd
data_swiss_boutiques = pd.read_excel('boutiques_in_switzerland.xlsx')

In [24]:
websites = data_swiss_boutiques['Website']

In [51]:
## Fetch emails from all of the websites

In [40]:
import requests
from bs4 import BeautifulSoup
import re
from urllib.parse import urlparse, urljoin
import threading

BASE_URLS = websites[len(emails_list):]

class EmailFoundException(Exception):
    """Exception raised when an email is found."""
    pass


def is_relevant_link(base_url, href):
    """Determine if the link is relevant for our search."""

    # Exclude specific file types
    ignored_extensions = ['.jpg', '.jpeg', '.png', '.css', '.js', '.gif', '.svg', '.webp']
    if any(href.endswith(ext) for ext in ignored_extensions):
        return False

    # Limit to same domain
    parsed_base = urlparse(base_url)
    parsed_href = urlparse(href)
    if parsed_href.netloc and parsed_base.netloc != parsed_href.netloc:
        return False

    # Exclude certain URL patterns
    ignored_patterns = ["/api/", "/assets/"]
    if any(pattern in href for pattern in ignored_patterns):
        return False

    return True


def find_emails_on_page(base_url, url, visited_urls, depth=0, max_depth=3):
    # If we've visited this URL before, skip it
    if url in visited_urls:
        return None

    # Add the URL to our set of visited URLs
    visited_urls.add(url)

    try:
        # Fetch the content of the page
        response = requests.get(url)
        response.raise_for_status()  # raise exception for bad responses
        soup = BeautifulSoup(response.content, 'html.parser')

        # Extract emails with regex
        email_pattern = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,3}")
        page_emails = email_pattern.findall(soup.text)
        if page_emails:
            raise EmailFoundException(page_emails[0])  # raise exception with the found email

        # Limit the depth of the recursion to avoid getting stuck in loops
        if depth > max_depth:
            return None

        # Find and visit all internal links
        for a_tag in soup.find_all('a', href=True):
            href = a_tag['href']

            # Check if the link is relevant for our search
            if not is_relevant_link(base_url, href):
                continue

            # Check if href starts with http or https
            if href.startswith('http'):
                find_emails_on_page(base_url, href, visited_urls, depth=depth+1)
            else:
                # Construct full URL for relative paths
                new_url = urljoin(base_url, href)
                find_emails_on_page(base_url, new_url, visited_urls, depth=depth+1)

    except EmailFoundException as e:
        raise  # Propagate the exception upwards
    except Exception as e:
        print(f"Error while processing {url}: {e}")
        return None


def process_website(url):
    """Process a website to find an email within a certain timeframe."""

    # Store the result of the email scraping in a list for thread-safe operations
    result = []

    def worker():
        try:
            email = find_emails_on_page(url, url, set())
            if email:
                result.append(email)
        except EmailFoundException as e:
            result.append(e.args[0])

    # Create a thread for the email scraping process
    thread = threading.Thread(target=worker)
    thread.start()

    # Wait for 7-8 seconds for the thread to complete
    thread.join(10)  # You can adjust this value as needed

    # If the thread is still alive after the timeout, it means it's still working and we can stop it and move on
    if thread.is_alive():
        print(f"Timeout reached for {url}. Moving to next website.")
        return None  # or return a placeholder message if desired

    return result[0] if result else None


# Example usage:
# emails_list = []
for url in BASE_URLS:
    email = process_website(url)
    if email:
        print(f"Found email: {email} on {url}")
        emails_list.append(email)
    else:
        emails_list.append('None')

print("Finished processing all websites.")


Found email: mail@aare-boutique.chW on http://www.aare-boutique.ch/
Timeout reached for https://www.bijoulesboutiques.ch/. Moving to next website.
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Found email: info@cachet.ch on http://www.cachet.ch/
Found email: nsingenprisca.blunschi@quickline.ch on http://www.butig.ch/
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing javascript:;: No connection adapters were found for 'javascript:;'
Found email: nizboutique@colora-koeniz.ch on http://www.colora-koeniz.ch/
Error while processing http://jutta-bern.ch/: 403 Client Error: Forbidden for url: http://jutta-bern.ch/
Found email: Leila@romanceliving.ch on http://www.romanceliving.ch

Error while processing https://finde-offen.ch/buelach/b/coop: 404 Client Error: Not Found for url: https://finde-offen.ch/buelach/b/coop
Error while processing https://finde-offen.ch/buelach/b/raiffeisen: 404 Client Error: Not Found for url: https://finde-offen.ch/buelach/b/raiffeisen
Error while processing https://finde-offen.ch/buelach/b/swiss-post: 404 Client Error: Not Found for url: https://finde-offen.ch/buelach/b/swiss-post
Timeout reached for https://finde-offen.ch/buelach/fan-moda-mathewos-beyene-9516. Moving to next website.
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Found email: info@modetor.ch on http://www.modetor.ch/
Found email: 50info@paradisdesinnocents.ch on https://paradisdesinnocents.ch/
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing http://www.makingthings.ch/: 520 Server Error:  for url: https://www.makingthings.ch/
Found email: madame@m

Error while processing http://www.thestories.ch/collections/pottery-studio: 430 Client Error:  for url: http://www.thestories.ch/collections/pottery-studio
Error while processing http://www.thestories.ch/collections/workshops: 430 Client Error:  for url: http://www.thestories.ch/collections/workshops
Error while processing https://www.thestories.ch/blogs/news/tagged/events: 430 Client Error:  for url: https://www.thestories.ch/blogs/news/tagged/events
Timeout reached for https://chicoree.ch/de/filialen/luzern-hertensteinstrasse-hertensteinstrasse-2931-lu-6004/id/138/?utm_source=google-local&utm_medium=organic&utm_campaign=Google+Business+Profile+%2810011%29. Moving to next website.
Error while processing http://www.thestories.ch/blogs/news: 430 Client Error:  for url: http://www.thestories.ch/blogs/news
Error while processing https://www.thestories.ch/collections/pearls-shells: 430 Client Error:  for url: https://www.thestories.ch/collections/pearls-shells
Error while processing https:

Error while processing http://www.thestories.ch/collections/all-products/products/dream: 430 Client Error:  for url: http://www.thestories.ch/collections/all-products/products/dream
Error while processing http://www.thestories.ch/collections/all-products/products/mini-heirloom-claw-in-blonde-tortoise: 430 Client Error:  for url: http://www.thestories.ch/collections/all-products/products/mini-heirloom-claw-in-blonde-tortoise
Error while processing http://www.thestories.ch/collections/all-products/products/grande-heirloom-claw-in-blonde-tortoise: 430 Client Error:  for url: http://www.thestories.ch/collections/all-products/products/grande-heirloom-claw-in-blonde-tortoise
Error while processing http://www.thestories.ch/collections/all-products/products/cosmic-affirmation: 430 Client Error:  for url: http://www.thestories.ch/collections/all-products/products/cosmic-affirmation
Error while processing http://www.thestories.ch/collections/all-products/products/you-wonderful-creature-card: 430

Error while processing javascript:void(0);: No connection adapters were found for 'javascript:void(0);'
Error while processing mailto:info@kaufbar-teufen.ch: No connection adapters were found for 'mailto:info@kaufbar-teufen.ch'
Error while processing https://boutique-dress-for-you.business.site/: 503 Server Error: Service Unavailable for url: https://boutique-dress-for-you.business.site/
Timeout reached for https://yellow.local.ch/d/ASC01xPhINso3L6sOUuFlg. Moving to next website.
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing http://boutique-creative.ch/content/128/60/impressum/agb: HTTPConnectionPool(host='boutique-creative.ch', port=80): Max retries exceeded with url: /content/128/60/impressum/agb (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x118f914d0>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Found email: info@art-emoda.ch 

...

Error while processing https://www.tally-weijl.com/de_DE/unidays-studentenrabatt: 404 Client Error: Not Found for url: https://www.tally-weijl.com/de_DE/unidays-studentenrabatt
Found email: 17avenches@zanotta-mode.ch on https://www.zanotta-mode.ch/
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing http://www.angeloz-mode.ch/: HTTPConnectionPool(host='www.angeloz-mode.ch', port=80): Max retries exceeded with url: / (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x11dba0590>: Failed to establish a new connection: [Errno 8] nodename nor servname provided, or not known'))
Error while processing https://stores.maison123.com/fr_CH/123-manor-fribourg.html?utm_campaign=google_my_business&utm_medium=organic&utm_source=google: HTTPSConnectionPool(host='stores.maison123.com', port=443): Max retries exceeded with url: /fr_CH/123-manor-fribourg.html?utm_campaign=google_my_business&utm_medium=organic&

Error while processing http://www.wabisabi.earth/images/1FDD2B60-F3A7-450C-A7BB-70343D327C5C.jpg?asset_id=CE135205-A43E-4BC6-8642-1C9BFAD5904F&size=1024: 404 Client Error: Not Found for url: https://www.wabisabi.earth/images/1FDD2B60-F3A7-450C-A7BB-70343D327C5C.jpg?asset_id=CE135205-A43E-4BC6-8642-1C9BFAD5904F&size=1024
Error while processing http://www.wabisabi.earth/images/884E1407-4068-4E09-A009-53EB1FF17AB7.jpg?asset_id=FA759DAD-64A8-4BC6-8A85-D6DF936A9189&size=1024: 404 Client Error: Not Found for url: https://www.wabisabi.earth/images/884E1407-4068-4E09-A009-53EB1FF17AB7.jpg?asset_id=FA759DAD-64A8-4BC6-8A85-D6DF936A9189&size=1024
Error while processing http://www.wabisabi.earth/images/935EBC0A-9BBF-4BBF-B3A9-2FA945556E14.jpg?asset_id=284A8EB2-3CAA-4F7F-87C7-9F6B2FD71E2A&size=1024: 404 Client Error: Not Found for url: https://www.wabisabi.earth/images/935EBC0A-9BBF-4BBF-B3A9-2FA945556E14.jpg?asset_id=284A8EB2-3CAA-4F7F-87C7-9F6B2FD71E2A&size=1024
Error while processing https://www

Error while processing http://www.ilregalino.ch/products/tenerezze-spotty-100-g-sale: 430 Client Error:  for url: http://www.ilregalino.ch/products/tenerezze-spotty-100-g-sale
Timeout reached for https://yellow.local.ch/d/rBYmrLOpd5JjxLFIbVXDFw. Moving to next website.
Error while processing http://www.ilregalino.ch/products/dragees-perlen-grun-100-g-sale: 430 Client Error:  for url: http://www.ilregalino.ch/products/dragees-perlen-grun-100-g-sale
Error while processing http://www.ilregalino.ch/products/zuckermandeln-perlati-rosa-100-g-sale: 430 Client Error:  for url: http://www.ilregalino.ch/products/zuckermandeln-perlati-rosa-100-g-sale
Error while processing http://www.wabisabi.earth/images/057A7C7E-B6B5-465B-B6E6-EE4336418C43.jpg?asset_id=9EB98586-77E2-490B-9EDB-1C1B226ECB6C&size=1024: 404 Client Error: Not Found for url: https://www.wabisabi.earth/images/057A7C7E-B6B5-465B-B6E6-EE4336418C43.jpg?asset_id=9EB98586-77E2-490B-9EDB-1C1B226ECB6C&size=1024
Error while processing http://

Error while processing http://www.wabisabi.earth/images/51C14AE6-32E4-4E78-8C2F-8E33360CF60E.jpg?asset_id=38BABB90-165A-46E3-B839-7F5344FDD1EA&size=1024: 404 Client Error: Not Found for url: https://www.wabisabi.earth/images/51C14AE6-32E4-4E78-8C2F-8E33360CF60E.jpg?asset_id=38BABB90-165A-46E3-B839-7F5344FDD1EA&size=1024
Error while processing javascript:void(0): No connection adapters were found for 'javascript:void(0)'
Error while processing http://www.wabisabi.earth/images/E066BF95-64D3-49A4-8A0F-4B830ACE56F6.jpg?asset_id=141D1405-CF89-4D74-84DC-FD4FF196F3DF&size=1024: 404 Client Error: Not Found for url: https://www.wabisabi.earth/images/E066BF95-64D3-49A4-8A0F-4B830ACE56F6.jpg?asset_id=141D1405-CF89-4D74-84DC-FD4FF196F3DF&size=1024
Error while processing http://www.wabisabi.earth/images/B5708F9B-2E2F-4B03-A3EC-121CFB15BEAC.jpg?asset_id=0B3068DC-DA85-4438-9B9C-F41B77452F60&size=1024: 404 Client Error: Not Found for url: https://www.wabisabi.earth/images/B5708F9B-2E2F-4B03-A3EC-121CF

Timeout reached for http://www.ac-crea-design.ch/. Moving to next website.
Error while processing mailto:info@ac-crea-design.ch: No connection adapters were found for 'mailto:info@ac-crea-design.ch'
Error while processing tel:+41 27 288 50 00: No connection adapters were found for 'tel:+41 27 288 50 00'
Found email: 23Emailinfo@froufrou.chA on https://yellow.local.ch/d/6x7Chw-7k-TytnT7eW1c6g
Timeout reached for http://kanoaitalia.com/. Moving to next website.
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Timeout reached for http://www.calida.com/. Moving to next website.
Error while processing nan: Invalid URL 'nan': No scheme supplied. Perhaps you meant https://nan?
Error while processing tel:0041217280660: No connection adapters were found for 'tel:0041217280660'
Error while processing mailto:nathalie.opj@gmail.com?subject=Nouveau%20mes

In [44]:
data_swiss_boutiques['emails'] = emails_list

In [46]:
data_swiss_boutiques.to_excel('swiss_boutiques_with_emails.xlsx')

Error while processing javascript:$('#form_article_2767').submit(): No connection adapters were found for "javascript:$('#form_article_2767').submit()"
Error while processing javascript:$('#form_article_2752').submit(): No connection adapters were found for "javascript:$('#form_article_2752').submit()"
Error while processing javascript:$('#form_article_2792').submit(): No connection adapters were found for "javascript:$('#form_article_2792').submit()"
Error while processing javascript:$('#form_article_2787').submit(): No connection adapters were found for "javascript:$('#form_article_2787').submit()"
Error while processing javascript:$('#form_article_2782').submit(): No connection adapters were found for "javascript:$('#form_article_2782').submit()"
Error while processing javascript:$('#form_article_2777').submit(): No connection adapters were found for "javascript:$('#form_article_2777').submit()"
Error while processing javascript:$('#form_article_2742').submit(): No connection adapte

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Error while processing javascript:void(0): No connection adapters were found for 'javascript:void(0)'
Error while processing https://www.calida.com/Widerrufsrecht-calida/: 404 Client Error: Not Found for url: https://www.calida.com/de-WO/Widerrufsrecht-calida/
Error while processing https://www.iwc.com/en/pos/boutiques/iwc-schaffhausen-boutique-schaffhausen.html?y_source=1_NzI5MTQyNy03MTUtbG9jYXRpb24ud2Vic2l0ZQ%3D%3D: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response'))
Error while processing https://www.wefashion.ch/de_CH/kinder/m%C3%A4dchen/sweatshirts/kapuzensweatshirts%0A/: 404 Client Error: Not Found for url: https://www.wefashion.ch/de_CH/kinder/m%C3%A4dchen/sweatshirts/kapuzensweatshirts%0A/
Error while processing https://www.wefashion.ch/de_CH/kinder/m%C3%A4dchen/sweatshirts/sweatshirts-mit-rundhalsausschnitt%0A/: 404 Client Error: Not Found for url: https://www.wefashion.ch/de_CH/kinder/m%C3%A4dchen/sweatshirts/sweatshirts-mit-rundhalsau